In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib

In [2]:
df = pd.read_csv('../Dataset/processed_data/final_dataset.csv')

descriptions = df['description']  
domains = df['domain'] 
sub_domains = df['sub_domain']  

In [3]:

# Tokenize descriptions
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(descriptions)
X = tokenizer.texts_to_sequences(descriptions)
X = pad_sequences(X, padding='post', maxlen=100)  # Adjust maxlen as needed


In [4]:

# Encode labels (domain and sub_domain)
domain_encoder = joblib.load("../models/domain_label_encoder.pkl")
sub_domain_encoder = joblib.load("../models/sub_domain_label_encoder.pkl")
y_domain = domain_encoder.transform(domains)
y_sub_domain = sub_domain_encoder.transform(sub_domains)

# One-hot encode labels
y_domain = tf.keras.utils.to_categorical(y_domain, num_classes=len(domain_encoder.classes_))
y_sub_domain = tf.keras.utils.to_categorical(y_sub_domain, num_classes=len(sub_domain_encoder.classes_))


# Split data into train (80%) and validation (20%)
X_train, X_val, y_domain_train, y_domain_val, y_sub_train, y_sub_val = train_test_split(
    X, y_domain, y_sub_domain, test_size=0.2, random_state=42
)

In [6]:

# Build Neural Network Model
input_layer = Input(shape=(X.shape[1],))
embedding_layer = Embedding(input_dim=10000, output_dim=128, input_length=X.shape[1])(input_layer)
lstm_layer = LSTM(64, return_sequences=False)(embedding_layer)
dropout_layer = Dropout(0.5)(lstm_layer)
dense_layer = Dense(64, activation='relu')(dropout_layer)

# Domain Output Layer
domain_output = Dense(len(domain_encoder.classes_), activation='softmax', name='domain')(dense_layer)

# Sub-Domain Output Layer
sub_domain_output = Dense(len(sub_domain_encoder.classes_), activation='softmax', name='sub_domain')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=[domain_output, sub_domain_output])


d:\Jay\CDAC\Project\Domain Classifier\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:
# Compile the model with separate metrics for each output
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'categorical_crossentropy'],
              metrics=[['accuracy'], ['accuracy']])  

# Summary of the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 128)  │  1,280,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     49,408 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ domain (Dense)      │ (None, 4)         │        260 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sub_domain (Dense)  │ (None, 46)        │      2,990 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,336,818 (5.10 MB)

 Trainable params: 1,336,818 (5.10 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:

# Train the model
history = model.fit(
    X_train, [y_domain_train, y_sub_train],
    epochs=20,
    batch_size=16,
    validation_data=(X_val, [y_domain_val, y_sub_val]),
    verbose=1
)

Epoch 1/20
8249/8249 ━━━━━━━━━━━━━━━━━━━━ 798s 95ms/step - domain_accuracy: 0.7873 - domain_loss: 0.5174 - loss: 3.1405 - sub_domain_accuracy: 0.1764 - sub_domain_loss: 2.6231 - val_domain_accuracy: 0.9871 - val_domain_loss: 0.0457 - val_loss: 1.5264 - val_sub_domain_accuracy: 0.3817 - val_sub_domain_loss: 1.4803
Epoch 2/20
8249/8249 ━━━━━━━━━━━━━━━━━━━━ 871s 106ms/step - domain_accuracy: 0.9885 - domain_loss: 0.0434 - loss: 1.3923 - sub_domain_accuracy: 0.4737 - sub_domain_loss: 1.3489 - val_domain_accuracy: 0.9925 - val_domain_loss: 0.0265 - val_loss: 0.7530 - val_sub_domain_accuracy: 0.7406 - val_sub_domain_loss: 0.7262
Epoch 3/20
8249/8249 ━━━━━━━━━━━━━━━━━━━━ 771s 93ms/step - domain_accuracy: 0.9923 - domain_loss: 0.0286 - loss: 0.7446 - sub_domain_accuracy: 0.7443 - sub_domain_loss: 0.7160 - val_domain_accuracy: 0.9903 - val_domain_loss: 0.0394 - val_loss: 0.6259 - val_sub_domain_accuracy: 0.8087 - val_sub_domain_loss: 0.5863
Epoch 4/20
8249/8249 ━━━━━━━━━━━━━━━━━━━━ 643s 78ms/st

In [9]:
# Evaluate the model on validation set
val_loss, val_domain_loss, val_sub_loss, val_domain_acc, val_sub_acc = model.evaluate(
    X_val, [y_domain_val, y_sub_val])
print(
    f"Validation Accuracy - Domain: {val_domain_acc:.4f}, Sub-Domain: {val_sub_acc:.4f}")

1032/1032 ━━━━━━━━━━━━━━━━━━━━ 31s 30ms/step - domain_accuracy: 0.9926 - domain_loss: 0.0487 - loss: 0.8517 - sub_domain_accuracy: 0.8661 - sub_domain_loss: 0.8030
Validation Accuracy - Domain: 0.9927, Sub-Domain: 0.8641


In [10]:
description_input = ["""Battery Details- Type: Tall tubular batteryCapacity : 200Ah/12V: Construction: rugged construction
Warranty -66 months ( 48 FOC+18Pro Rata)
Dimension (in cm) - 512x192x466
Weight- 64.8K.G"""]
description_seq = tokenizer.texts_to_sequences(description_input)
description_padded = pad_sequences(description_seq, padding='post', maxlen=100)

domain_pred, sub_domain_pred = model.predict(description_padded)
domain_pred_label = domain_encoder.inverse_transform(
    domain_pred.argmax(axis=1))
sub_domain_pred_label = sub_domain_encoder.inverse_transform(
    sub_domain_pred.argmax(axis=1))

print(f"Predicted Domain: {domain_pred_label}")
print(f"Predicted Sub-Domain: {sub_domain_pred_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Domain: ['Ecommerce']
Predicted Sub-Domain: ['Household']


In [11]:
# Save the entire model (architecture + weights + optimizer state)
import pickle

# Save the model
model.save("../models/LSTM/multi_label_model.keras")

# Save the tokenizer
joblib.dump(tokenizer, "../models/LSTM/tokenizer.pkl")

print("Model and necessary objects saved successfully! 🎯")

Model and necessary objects saved successfully! 🎯


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# Load the saved model
model = load_model("../models/LSTM/multi_label_model.h5")

# Load Tokenizer
tokenizer = joblib.load("../models/LSTM/tokenizer.pkl")

# Load Label Encoders
domain_encoder = joblib.load("../models/domain_label_encoder.pkl")


sub_domain_encoder = joblib.load("../models/sub_domain_label_encoder.pkl")

print("Model and necessary objects loaded successfully! 🚀")